## I. Setup & Data Loading

In [ ]:
# Install required packages
%pip install lightgbm shap scikit-learn --quiet

In [ ]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
import shap

# Set styles
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
pd.set_option('display.max_columns', None)

print("Libraries imported successfully")

In [ ]:
# Load turbine features
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
FEATURES_DIR = BASE_DIR / "data/features"
MODELS_DIR = BASE_DIR / "models"
PREDICTIONS_DIR = BASE_DIR / "predictions"

MODELS_DIR.mkdir(parents=True, exist_ok=True)
PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

# Load data
turbine_df = pd.read_csv(FEATURES_DIR / "turbine_features.csv")
turbine_df['timestamp'] = pd.to_datetime(turbine_df['timestamp'])

print(f"Dataset shape: {turbine_df.shape}")
print(f"Unique turbines: {turbine_df['equipment_id'].nunique()}")
print(f"\nDataset split:")
print(turbine_df['dataset'].value_counts())
print(f"\nRUL statistics:")
print(turbine_df['rul_actual'].describe())

## II. Feature Selection & Preprocessing

In [ ]:
# Select features for modeling
# Exclude: equipment_id, timestamp, dataset (split indicator), is_anomaly, metadata cols

feature_cols = [
    # Time/cycle indicator
    'time_cycles',
    
    # Operational settings (3)
    'op_setting_1', 'op_setting_2', 'op_setting_3',
    
    # Key sensors (7)
    'sensor_2', 'sensor_4', 'sensor_7', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_17',
    
    # Engineered features (4)
    'temp_gradient', 'pressure_ratio_norm', 'speed_ratio', 'efficiency_proxy',
    
    # Rolling statistics (4)
    'rolling_mean_sensor_2', 'rolling_std_sensor_2',
    'rolling_mean_sensor_11', 'rolling_std_sensor_11',
    
    # Degradation trend
    'temp_trend_slope',
    
    # Health index
    'health_index'
]

target_col = 'rul_actual'

print(f"Selected {len(feature_cols)} features for modeling")
print(f"\nFeature categories:")
print(f"  - Operational settings: 3")
print(f"  - Sensor readings: 7")
print(f"  - Engineered features: 4")
print(f"  - Rolling statistics: 4")
print(f"  - Degradation trends: 1")
print(f"  - Health index: 1")
print(f"  - Time cycles: 1")

In [ ]:
# Split data using existing train/test split from CMAPS dataset
train_data = turbine_df[turbine_df['dataset'] == 'train'].copy()
test_data = turbine_df[turbine_df['dataset'] == 'test'].copy()

print(f"Train set: {train_data.shape}")
print(f"Test set: {test_data.shape}")

# Prepare X and y
X_train = train_data[feature_cols]
y_train = train_data[target_col]
X_test = test_data[feature_cols]
y_test = test_data[target_col]

print(f"\nX_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

In [ ]:
# Check for missing values
print("Missing values in train set:")
print(X_train.isnull().sum()[X_train.isnull().sum() > 0])

print("\nMissing values in test set:")
print(X_test.isnull().sum()[X_test.isnull().sum() > 0])

# Fill missing values with forward fill (for rolling stats at start)
X_train = X_train.fillna(method='ffill').fillna(method='bfill')
X_test = X_test.fillna(method='ffill').fillna(method='bfill')

print("\nMissing values filled successfully")

## III. Model Training

In [ ]:
# LightGBM Regressor parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 42
}

# Create LightGBM datasets
train_dataset = lgb.Dataset(X_train, label=y_train)
test_dataset = lgb.Dataset(X_test, label=y_test, reference=train_dataset)

print("Training LightGBM Regressor...")
print(f"Parameters: {params}")

# Train model
evals_result = {}
model = lgb.train(
    params,
    train_dataset,
    num_boost_round=1000,
    valid_sets=[train_dataset, test_dataset],
    valid_names=['train', 'test'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100),
        lgb.record_evaluation(evals_result)
    ]
)

print(f"\n✓ Training complete. Best iteration: {model.best_iteration}")

In [ ]:
# Plot training history
fig, ax = plt.subplots(figsize=(12, 5))

train_rmse = evals_result['train']['rmse']
test_rmse = evals_result['test']['rmse']

ax.plot(train_rmse, label='Train RMSE', linewidth=2)
ax.plot(test_rmse, label='Test RMSE', linewidth=2)
ax.set_xlabel('Iteration')
ax.set_ylabel('RMSE')
ax.set_title('LightGBM Training History', fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final Train RMSE: {train_rmse[-1]:.2f}")
print(f"Final Test RMSE: {test_rmse[-1]:.2f}")

## IV. Model Evaluation

In [ ]:
# Make predictions
y_train_pred = model.predict(X_train, num_iteration=model.best_iteration)
y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Calculate metrics
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)
train_mape = np.mean(np.abs((y_train - y_train_pred) / (y_train + 1))) * 100

test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)
test_mape = np.mean(np.abs((y_test - y_test_pred) / (y_test + 1))) * 100

print("="*70)
print(" "*20 + "MODEL EVALUATION METRICS")
print("="*70)

print("\nTrain Set:")
print(f"  MAE:  {train_mae:.2f} cycles")
print(f"  RMSE: {train_rmse:.2f} cycles")
print(f"  R²:   {train_r2:.4f}")
print(f"  MAPE: {train_mape:.2f}%")

print("\nTest Set:")
print(f"  MAE:  {test_mae:.2f} cycles")
print(f"  RMSE: {test_rmse:.2f} cycles")
print(f"  R²:   {test_r2:.4f}")
print(f"  MAPE: {test_mape:.2f}%")

print("\n" + "="*70)

In [ ]:
# Visualization: Actual vs Predicted
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Train set
axes[0].scatter(y_train, y_train_pred, alpha=0.3, s=10)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 
             'r--', linewidth=2, label='Perfect prediction')
axes[0].set_xlabel('Actual RUL (cycles)')
axes[0].set_ylabel('Predicted RUL (cycles)')
axes[0].set_title(f'Train Set (R²={train_r2:.4f})', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Test set
axes[1].scatter(y_test, y_test_pred, alpha=0.3, s=10, color='orange')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', linewidth=2, label='Perfect prediction')
axes[1].set_xlabel('Actual RUL (cycles)')
axes[1].set_ylabel('Predicted RUL (cycles)')
axes[1].set_title(f'Test Set (R²={test_r2:.4f})', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis
train_residuals = y_train - y_train_pred
test_residuals = y_test - y_test_pred

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Train residuals distribution
axes[0, 0].hist(train_residuals, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Train Residuals Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Residual (Actual - Predicted)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(0, color='red', linestyle='--', linewidth=2)

# Test residuals distribution
axes[0, 1].hist(test_residuals, bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_title('Test Residuals Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Residual (Actual - Predicted)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=2)

# Train residuals vs predicted
axes[1, 0].scatter(y_train_pred, train_residuals, alpha=0.3, s=10)
axes[1, 0].set_title('Train Residuals vs Predicted', fontweight='bold')
axes[1, 0].set_xlabel('Predicted RUL')
axes[1, 0].set_ylabel('Residual')
axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1, 0].grid(alpha=0.3)

# Test residuals vs predicted
axes[1, 1].scatter(y_test_pred, test_residuals, alpha=0.3, s=10, color='orange')
axes[1, 1].set_title('Test Residuals vs Predicted', fontweight='bold')
axes[1, 1].set_xlabel('Predicted RUL')
axes[1, 1].set_ylabel('Residual')
axes[1, 1].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Residual Statistics (Test Set):")
print(f"  Mean: {test_residuals.mean():.2f} cycles")
print(f"  Std:  {test_residuals.std():.2f} cycles")
print(f"  Within ±20 cycles: {(np.abs(test_residuals) <= 20).sum() / len(test_residuals) * 100:.1f}%")
print(f"  Within ±50 cycles: {(np.abs(test_residuals) <= 50).sum() / len(test_residuals) * 100:.1f}%")

## V. Feature Importance & SHAP Analysis

In [ ]:
# Feature importance from LightGBM
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

print("Top 10 Features by Gain:")
print(feature_importance.head(10).to_string(index=False))

# Plot feature importance
fig, ax = plt.subplots(figsize=(10, 8))
top_features = feature_importance.head(15)
ax.barh(range(len(top_features)), top_features['importance'], color='steelblue')
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['feature'])
ax.set_xlabel('Feature Importance (Gain)')
ax.set_title('Top 15 Features by Importance', fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# SHAP analysis (sample 1000 for performance)
print("Computing SHAP values (sampling 1000 records)...")
sample_indices = np.random.choice(X_test.index, size=min(1000, len(X_test)), replace=False)
X_sample = X_test.loc[sample_indices]

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_sample)

print(" SHAP values computed")

In [ ]:
# SHAP summary plot
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False)
plt.title('SHAP Feature Importance (Mean |SHAP value|)', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# SHAP beeswarm plot
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_sample, show=False)
plt.title('SHAP Summary Plot (Impact on RUL Prediction)', fontweight='bold')
plt.tight_layout()
plt.show()

## VI. RUL Predictions & Maintenance Scheduling

In [ ]:
# Create predictions dataframe
test_predictions = test_data.copy()
test_predictions['rul_predicted'] = y_test_pred
test_predictions['rul_error'] = test_predictions['rul_actual'] - test_predictions['rul_predicted']
test_predictions['rul_error_abs'] = np.abs(test_predictions['rul_error'])

# Convert RUL to days (assuming 1 cycle = 1 hour flight)
test_predictions['rul_actual_days'] = test_predictions['rul_actual'] / 24.0
test_predictions['rul_predicted_days'] = test_predictions['rul_predicted'] / 24.0

print("Predictions created for test set")
print(test_predictions[['equipment_id', 'time_cycles', 'rul_actual', 'rul_predicted', 
                         'rul_error', 'health_index']].head(10))

In [ ]:
# Get latest predictions per turbine
latest_predictions = test_predictions.sort_values('time_cycles').groupby('equipment_id').last()

# Critical turbines (RUL < 50 cycles or ~2 days)
critical_turbines = latest_predictions[
    (latest_predictions['rul_predicted'] < 50) | 
    (latest_predictions['health_index'] < 0.5)
].copy()

critical_turbines = critical_turbines.sort_values('rul_predicted')

print("="*70)
print(" "*20 + "CRITICAL TURBINES")
print("="*70)
print(f"\nTotal critical turbines: {len(critical_turbines)}")

if len(critical_turbines) > 0:
    print("\nTop 10 turbines requiring immediate attention:")
    print(critical_turbines[['rul_predicted', 'rul_predicted_days', 'health_index']].head(10))
else:
    print("\n No critical turbines found")

In [ ]:
# Maintenance scheduling recommendations
latest_predictions['maintenance_priority'] = pd.cut(
    latest_predictions['rul_predicted'],
    bins=[-np.inf, 30, 60, 120, np.inf],
    labels=['Immediate (< 30 cycles)', 'Urgent (30-60 cycles)', 
            'Scheduled (60-120 cycles)', 'Normal (> 120 cycles)']
)

print("\nMaintenance Priority Distribution:")
priority_counts = latest_predictions['maintenance_priority'].value_counts().sort_index()
for priority, count in priority_counts.items():
    print(f"  {priority}: {count} turbines ({count/len(latest_predictions)*100:.1f}%)")

# Plot maintenance schedule
fig, ax = plt.subplots(figsize=(10, 6))
priority_counts.plot(kind='bar', ax=ax, color=['red', 'orange', 'yellow', 'green'])
ax.set_title('Maintenance Priority Distribution', fontweight='bold')
ax.set_xlabel('Priority Level')
ax.set_ylabel('Number of Turbines')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.tight_layout()
plt.show()

## VII. Model Export & Predictions Save

In [ ]:
# Save model
timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
model_path = MODELS_DIR / f"lgb_turbine_rul_model_{timestamp}.txt"
model.save_model(str(model_path))
print(f" Model saved to: {model_path}")

# Save predictions
predictions_path = PREDICTIONS_DIR / f"turbine_rul_predictions_{timestamp}.csv"
test_predictions.to_csv(predictions_path, index=False)
print(f" Predictions saved to: {predictions_path}")

# Save critical turbines
if len(critical_turbines) > 0:
    critical_path = PREDICTIONS_DIR / f"critical_turbines_{timestamp}.csv"
    critical_turbines.to_csv(critical_path)
    print(f" Critical turbines saved to: {critical_path}")

# Save maintenance schedule
maintenance_path = PREDICTIONS_DIR / f"turbine_maintenance_schedule_{timestamp}.csv"
latest_predictions[['rul_predicted', 'rul_predicted_days', 'health_index', 
                    'maintenance_priority']].to_csv(maintenance_path)
print(f" Maintenance schedule saved to: {maintenance_path}")

## VIII. Summary & Key Findings

In [ ]:
print("="*80)
print(" "*25 + "MODEL SUMMARY")
print("="*80)

print("\n1. MODEL PERFORMANCE:")
print(f"   - Test MAE:  {test_mae:.2f} cycles (~{test_mae/24:.1f} days)")
print(f"   - Test RMSE: {test_rmse:.2f} cycles (~{test_rmse/24:.1f} days)")
print(f"   - Test R²:   {test_r2:.4f}")
print(f"   - Predictions within ±20 cycles: {(np.abs(test_residuals) <= 20).sum() / len(test_residuals) * 100:.1f}%")

print("\n2. TOP 5 MOST IMPORTANT FEATURES:")
for idx, row in feature_importance.head(5).iterrows():
    print(f"   {idx+1}. {row['feature']}: {row['importance']:.0f}")

print("\n3. MAINTENANCE RECOMMENDATIONS:")
immediate = len(latest_predictions[latest_predictions['maintenance_priority'] == 'Immediate (< 30 cycles)'])
urgent = len(latest_predictions[latest_predictions['maintenance_priority'] == 'Urgent (30-60 cycles)'])
scheduled = len(latest_predictions[latest_predictions['maintenance_priority'] == 'Scheduled (60-120 cycles)'])

print(f"   - Immediate attention (< 30 cycles): {immediate} turbines")
print(f"   - Urgent maintenance (30-60 cycles): {urgent} turbines")
print(f"   - Scheduled maintenance (60-120 cycles): {scheduled} turbines")

print("\n4. KEY INSIGHTS:")
print(f"   - Health index is the strongest predictor (composite of efficiency, temp, pressure, speed)")
print(f"   - Time cycles and rolling statistics capture degradation trends effectively")
print(f"   - Temperature gradient (T30-T2) indicates engine thermal stress")
print(f"   - Model achieves R²={test_r2:.3f} on unseen test engines")

print("\n5. NEXT STEPS:")
print("   - Integrate predictions into real-time dashboard")
print("   - Set up automated alerts for turbines with RUL < 30 cycles")
print("   - Retrain model periodically with new operational data")
print("   - Consider LSTM for sequential time-series modeling")

print("\n" + "="*80)
print("TURBINE RUL MODELING COMPLETE")
print("="*80)